<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/SVM_Exact_Replication_Experiment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Data Science
import re
import csv
import json
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

# SVM
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [3]:
spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewSpiral.csv')

In [4]:
meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewSpiral.csv')

In [5]:
spiral_df['ID_PATIENT'][spiral_df['_ID_EXAM'] == "P26"] = 32
meander_df['ID_PATIENT'][meander_df['_ID_EXAM'] == "P26"] = 32

In [6]:
spiral_df.columns

Index(['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE', 'RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT'],
      dtype='object')

In [7]:
# extracting necessary columns from spiral df
X_spiral = spiral_df[['RMS', 'MAX_BETWEEN_ET_HT', 'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT', 'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]
y_spiral = spiral_df[['CLASS_TYPE']]

In [8]:
# extracting necessary columns from meander df
X_meander = meander_df[['RMS', 'MAX_BETWEEN_ET_HT', 'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT', 'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]
y_meander = meander_df[['CLASS_TYPE']]

##**Preprocessing**

###**Normalization**

fi' = (fi - avg)/std

In [9]:
# normalization with the paper's method (formula above)
def feature_normalization(df):
  avg_dev = df.mad(axis = 0)
  std_dev = df.std(axis = 0)

  df = df.sub(avg_dev)
  df = df.divide(std_dev)

  return df

In [10]:
# normalizing X_spiral
X_spiral = feature_normalization(X_spiral)

In [11]:
# normalizing X_meander
X_meander = feature_normalization(X_meander)

###Train-Test-Split

In [12]:
def data_split_experiment_3(X,y):# Train-Test Split, 80% train, 20% test, stratification across PD diagnosis (label)
  y_label = y.to_numpy()
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.25, stratify =y_label)

  X_train = X_train.reset_index(drop = True)
  X_test = X_test.reset_index(drop = True)  

  y_train = y_train.reset_index(drop = True)
  y_test = y_test.reset_index(drop = True)

  return X_train, X_test, y_train, y_test

1. try without stratified label perhaps because we want to avoid shuffle for experiment 3? even if it shuffles, it should choose the right patient since it shuffles together. It then resets for easiness, but right patient should be preserved.

In [13]:
X_spiral_train, X_spiral_test, y_spiral_train, y_spiral_test = data_split_experiment_3(X_spiral,y_spiral)

##**SVM Implementation**

In [33]:
score_arr = []

for i in range(20):

  X_spiral = feature_normalization(X_spiral)
  X_meander = feature_normalization(X_meander)

  X_meander_train, X_meander_test, y_meander_train, y_meander_test = data_split_experiment_3(X_meander,y_meander)
  X_spiral_train, X_spiral_test, y_spiral_train, y_spiral_test = data_split_experiment_3(X_spiral,y_spiral)

  clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced')
  clf.fit(X_meander_train, y_meander_train)

  meander_score = clf.score(X_meander_test, y_meander_test)

  y_pred = clf.predict(X_meander_test)
  meander_recall = recall_score(y_meander_test, y_pred)
  meander_precision = precision_score(y_meander_test, y_pred)


  clf1 = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced')
  clf1.fit(X_spiral_train, y_spiral_train)

  spiral_score = clf1.score(X_spiral_test, y_spiral_test)

  y_pred = clf1.predict(X_spiral_test)
  spiral_recall = recall_score(y_spiral_test, y_pred)
  spiral_precision = precision_score(y_spiral_test, y_pred)


  spiral_ov = (spiral_score + spiral_recall + spiral_precision)/3
  meander_ov = (meander_score + meander_recall + meander_precision)/3


  if (spiral_ov > meander_ov): # consider recall and such too? will there be one with highest recall, precision, and accuracy?
    score = spiral_score
    y_pred = clf.predict(X_spiral_test)
    conf_mat = confusion_matrix(y_spiral_test, y_pred)
  else:
    score = meander_score
    y_pred = clf.predict(X_meander_test)
    conf_mat = confusion_matrix(y_meander_test, y_pred)

  score_arr.append(score)
  print(conf_mat)
  print('')

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]

[[35  0]
 [31  0]]



In [34]:
score_arr

[0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303,
 0.5303030303030303]

In [32]:
np.mean(score_arr)

0.531818181818182

In [36]:
np.std(score_arr)

2.220446049250313e-16

The SVM approaches 53.03% always at some point. Perhaps there is a local minimum in the loss function it can't get out of?

Can we somehow get the weights for the model to learn which features are considered most useful?